### Настройки

In [ ]:
!pip install psycopg2

Начинаем с импорта библиотек

In [1]:
import pandas as pd
import psycopg2

In [2]:
DBNAME = ''
USER = ''
PASSWORD = ''
HOST = ''
PORT =

Создаем соединение с заданными параметрами

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

#Предварительный анализ данных

Количество вакансий

In [ ]:
pd.read_sql_query('select count(*) from vacancies', connection)

/tmp/ipython-input-3457821119.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('select count(*) from vacancies', connection)


,count
0,49197


Сколько вакансий есть в базе?

In [ ]:
pd.read_sql_query('select count(*) from employers', connection)

/tmp/ipython-input-1631717449.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('select count(*) from employers', connection)


,count
0,23501


Сколько регионов?

In [ ]:
pd.read_sql_query('select count(*) from areas', connection)

/tmp/ipython-input-3801176253.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('select count(*) from areas', connection)


,count
0,1362


Сколько сфер деятельности?

In [ ]:
pd.read_sql_query('select count(*) from industries', connection)

/tmp/ipython-input-3383566588.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('select count(*) from industries', connection)


,count
0,294


Основные выводы:

Объем данных: База содержит 49,197 вакансий и информацию о 13,614 работодателях

Данные охватывают 1,362 региона, включая международные рынки

Существует 294 различных сферы деятельности

# Детальный анализ вакансий

Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).

Выберите пятёрку лидеров по количеству вакансий:

In [ ]:
pd.read_sql_query('''
  select a.name as area
  ,count(v.id) AS cnt
  from vacancies v
  left join areas a
  on v.area_id = a.id
  group by a.name
  order by cnt desc
  limit 5
  ''', connection)

/tmp/ipython-input-831677187.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


Посмотрим на зарплаты. У какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой?

In [ ]:
pd.read_sql_query('''
  select count(*)
  from vacancies
  where 1 = 1
  and salary_from is not null
  or salary_to is not null
  ''', connection)

/tmp/ipython-input-2898734784.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,count
0,24073


Найдите средние значения для нижней и верхней границы зарплатной вилки.

Округлите значения до целого числа.

In [ ]:
pd.read_sql_query('''
  select
  round(avg(salary_from)) as avg_salary_min
  ,round(avg(salary_to)) as avg_salary_max
  from vacancies
  where 1 = 1
  and salary_from is not null
  or salary_to is not null
     ''', connection)

/tmp/ipython-input-3083547482.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,avg_salary_min,avg_salary_max
0,71065.0,110537.0


Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях.

Какая пара находится на втором месте по популярности?

In [ ]:
pd.read_sql_query('''
  select
  schedule
  ,employment
  ,count(*) as cnt
  from vacancies
  where 1 = 1
  and schedule is not null
  and employment is not null
  group by
  schedule
  ,employment
  order by cnt desc
  limit 2
     ''', connection)

/tmp/ipython-input-3656484152.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802


Напишите запрос, выводящий значения поля «Требуемый опыт работы» (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта.

In [ ]:
pd.read_sql_query('''
  select
  experience
  ,count(*) as cnt
  from vacancies
  where 1 = 1
  and experience is not null
  group by experience
  order by cnt
     ''', connection)

/tmp/ipython-input-3487912904.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,experience,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


Основные выводы:

Географическая концентрация: Москва (5,333 вакансий) и Санкт-Петербург (2,851) являются абсолютными лидерами

Минск (2,112) и Алматы (1,892) показывают активность на постсоветском пространстве

Только 24,073 вакансий (около 49%) содержат информацию о зарплате, что является ограничением для анализа компенсаций

Средняя нижняя граница - 71,065 руб., верхняя - 110,537 руб., что показывает значительный разброс в зарплатных ожиданиях

Полный день + полная занятость (35,367) - доминирующая модель

Наибольший спрос на специалистов с опытом 1-3 года (26,152), что указывает на потребность в mmiddle-специалистах

# Анализ работодателей

Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [ ]:
pd.read_sql_query('''
  select
    e.name as employer_name
    ,count(v.id) as vacancy_count
  from vacancies v
  join employers e on v.employer_id = e.id
  group by e.name
  order by vacancy_count desc
  limit 5
''', connection)

/tmp/ipython-input-2935680824.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,employer_name,vacancy_count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.

Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей. Впишите его название в поле ниже в том виде, который вернул запрос.

In [36]:
pd.read_sql_query('''
  select
  a.name
  ,count(v.id) as vacancies_count
  ,count(e.id) as employers_count
  from areas a
  left join vacancies v
  on a.id = v.area_id
  left join employers e
  on a.id = e.area
  where 1 = 1
  and v.id is null
  group by 1
  order by 3 desc
  limit 1
''', connection)

/tmp/ipython-input-3423723274.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,name,vacancies_count,employers_count
0,Россия,0,410


Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии.

Выберите максимальное значение из получившегося списка.

In [ ]:
pd.read_sql_query('''
  select
    e.name as employer_name
    ,count(distinct v.area_id) as regions_count
  from employers e
  join vacancies v on e.id = v.employer_id
  group by e.name
  order by regions_count desc
  limit 1
''', connection)

/tmp/ipython-input-3005074411.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,employer_name,regions_count
0,Яндекс,181


Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности.

In [ ]:
pd.read_sql_query('''
  select
  count(distinct e.id) as cnt
  from employers e
  where not exists (
    select 1
    from employers_industries ei
    where e.id = ei.employer_id
  )
''', connection)

/tmp/ipython-input-2628277405.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,cnt
0,8419


Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности.

Введите в поле ниже название этой компании так же, как оно указано в результате запроса.

In [ ]:
pd.read_sql_query('''
  select
  e.name
  from employers e
  join employers_industries ei
  on e.id = ei.employer_id
  group by e.name
  having count(ei.industry_id) = 4
  order by e.name
  limit 1
  offset 2
''', connection)

/tmp/ipython-input-1613373323.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,name
0,2ГИС


С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана «Разработка программного обеспечения».

In [ ]:
pd.read_sql_query('''
  select
  count(distinct e.id) as cnt
  from employers e
  inner join employers_industries ei
  on e.id = ei.employer_id
  inner join industries i
  on ei.industry_id = i.id
  and i.name = 'Разработка программного обеспечения'
''', connection)

/tmp/ipython-input-1715456772.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,cnt
0,3553


Для компании «Яндекс» выведите список городов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку "Total" с общим количеством вакансий компании, собранных в этой таблице

In [ ]:
pd.read_sql_query('''
with yv as (
    select
    a.name as city
    ,count(v.id) as cnt
    from vacancies v
    inner join employers e on v.employer_id = e.id
    inner join areas a on v.area_id = a.id
    and e.name = 'Яндекс'
    and a.name in ('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург',
                    'Нижний Новгород', 'Казань', 'Челябинск', 'Самара',
                    'Омск', 'Ростов-на-Дону', 'Уфа', 'Красноярск',
                    'Воронеж', 'Пермь', 'Волгоград', 'Краснодар')
    group by a.name
  )

  select * from yv

  union all

  select
  'Total' as city
  ,sum(cnt) as cnt
  from yv
''', connection)

/tmp/ipython-input-229974756.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,city,cnt
0,Волгоград,24.0
1,Воронеж,32.0
2,Екатеринбург,39.0
3,Казань,25.0
4,Краснодар,30.0
5,Красноярск,23.0
6,Москва,54.0
7,Нижний Новгород,36.0
8,Новосибирск,35.0
9,Омск,21.0


Основные выводы:

Яндекс (1,933 вакансий) значительно опережает других работодателей

Яндекс представлен в 181 регионе, что показывает максимальный географический охват среди всех работодателей

8,419 работодателей не указали сферу деятельности

Компании типа "2ГИС" работают в 4 сферах одновременно

Яндекс активно нанимает в городах-миллионниках, с лидерами по вакансиям: Москва (54), Санкт-Петербург (42), Екатеринбург (39)

# Предметный анализ

Сколько вакансий имеет отношение к данным?

In [ ]:
pd.read_sql_query('''
  select
  count(*) as cnt
  from vacancies
  where 1 = 1
  and lower(name) like '%data%'
  or lower(name) like '%данн%'
''', connection)

/tmp/ipython-input-4146520662.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,cnt
0,1771


Сколько есть подходящих вакансий для начинающего дата-сайентиста?

Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:

‘data scientist’;
‘data science’;
‘исследователь данных’;
‘ML’ (здесь не нужно брать вакансии по HTML);
‘machine learning’;
‘машинн%обучен%’.
В следующих заданиях мы продолжим работать с вакансиями по указанному выше условию (уже не учитывая вакансии уровня Junior).

Считаем вакансиями для специалистов уровня Junior следующие:

в названии есть слово “junior” или
требуемый опыт — «Нет опыта» или
тип трудоустройства — «Стажировка».

In [ ]:
pd.read_sql_query('''
  select
  count(*) as cnt
  from (
    select *
    from vacancies
    where lower(name) like '%data scientist%'
       or lower(name) like '%data science%'
       or lower(name) like '%исследователь данных%'
       or (lower(name) like '%ml%' and lower(name) not like '%html%')
       or lower(name) like '%machine learning%'
       or lower(name) like '%машинн%обучен%'
  ) t
  where 1 = 1
  and lower(name) like '%junior%' or experience = 'Нет опыта' or employment = 'Стажировка'
''', connection)

/tmp/ipython-input-2979926522.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,cnt
0,51


Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или Postgres?

In [ ]:
pd.read_sql_query('''
  select
    count(*) as cnt
  from (
    select *
    from vacancies
    where lower(name) like '%data scientist%'
       or lower(name) like '%data science%'
       or lower(name) like '%исследователь данных%'
       or (lower(name) like '%ml%' and lower(name) not like '%html%')
       or lower(name) like '%machine learning%'
       or lower(name) like '%машинн%обучен%'
  ) t
  where 1 = 1
  and (lower(key_skills) like '%sql%' or lower(key_skills) like '%postgres%')
''', connection)

/tmp/ipython-input-1454180628.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,cnt
0,229


С помощью запроса, аналогичного предыдущему, проверьте, насколько популярен Python в требованиях работодателей к DS. Вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

In [ ]:
pd.read_sql_query('''
  with ds_vacancies as (
    select *
    from vacancies
    where lower(name) like '%data scientist%'
       or lower(name) like '%data science%'
       or lower(name) like '%исследователь данных%'
       or (lower(name) like '%ml%' and lower(name) not like '%html%')
       or lower(name) like '%machine learning%'
       or lower(name) like '%машинн%обучен%'
  )
  select
    count(*) as cnt
  from (
    select *
    from vacancies
    where lower(name) like '%data scientist%'
       or lower(name) like '%data science%'
       or lower(name) like '%исследователь данных%'
       or (lower(name) like '%ml%' and lower(name) not like '%html%')
       or lower(name) like '%machine learning%'
       or lower(name) like '%машинн%обучен%'
  ) t
  where 1 = 1
  and lower(key_skills) like '%python%'
''', connection)

/tmp/ipython-input-3149405198.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,cnt
0,357


Сколько ключевых навыков в среднем указывают в вакансиях для DS?

In [ ]:
pd.read_sql_query('''
  select
    round(avg(case
      when key_skills is null or trim(key_skills) = '' then 0
      else array_length(string_to_array(nullif(trim(key_skills), ''), ','), 1)
    end), 2) as avg_skills_per_vacancy
  from (
    select *
    from vacancies
    where 1 = 1
    and key_skills is not null
    and (lower(name) like '%data scientist%'
      or lower(name) like '%data science%'
      or lower(name) like '%исследователь данных%'
      or (lower(name) like '%ml%' and lower(name) not like '%html%')
      or lower(name) like '%machine learning%'
      or lower(name) like '%машинн%обучен%')
  ) t
''', connection)

/tmp/ipython-input-2916317659.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,avg_skills_per_vacancy
0,1.09


Напишите запрос, позволяющий вычислить, какую зарплату для DS в среднем указывают для каждого типа требуемого опыта (уникальное значение из поля experience).
Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа.

In [4]:
pd.read_sql_query('''
  select
    experience,
    round(avg(case
      when salary_from is not null and salary_to is not null then (salary_from + salary_to) / 2
      when salary_from is not null then salary_from
      else salary_to
    end)) as avg_salary
  from vacancies
  where 1 = 1
    and (salary_from is not null or salary_to is not null)
    and (lower(name) like '%data scientist%'
      or lower(name) like '%data science%'
      or lower(name) like '%исследователь данных%'
      or (lower(name) like '%ml%' and lower(name) not like '%html%')
      or lower(name) like '%machine learning%'
      or lower(name) like '%машинн%обучен%')
  group by experience
  order by experience;
''', connection)

/tmp/ipython-input-2509691938.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query('''


,experience,avg_salary
0,Более 6 лет,157933.0
1,Нет опыта,74643.0
2,От 1 года до 3 лет,144233.0
3,От 3 до 6 лет,256454.0


Основные выводы:

1,771 вакансий связано с данными, что составляет около 3.6% от общего числа, показывая нишевый, но важный сегмент

Только 51 вакансия для Junior DS, что указывает на высокий барьер входа в профессию

SQL/Postgres требуется в 229 вакансиях (около 13% от DS-вакансий), Python - в 357 (около 20%)

Четкая градация - от 74,643 руб. для начинающих до 256,454 руб. для специалистов с опытом 3-6 лет

### Финал

Общий вывод по результатам анализа:

1. Рынок IT-вакансий в России значительный - почти 50,000 вакансий от более чем 13,000 работодателей, сильная концентрация в Москве и Санкт-Петербурге (около 16.6% всех вакансий)

2. Data Science как направления составляет небольшой, но важный сегмент рынка (около 3.6%). Небольшое количество Junior-вакансий (51) указывает на высокий барьер входа. Python и SQL являются обязательными требованиями. Четкая зависимость зарплаты от опыта, с пиком для специалистов 3-6 лет

3. Большой объем данных, широкий географический и отраслевой охват

In [37]:
# не забываем закрыть соединение после окончания работы
connection.close()